In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
import json
import math
import re
import ast
from collections import Counter

In [8]:
import pandas as pd
## csv 확인

# 1. 파일 읽기
df_check = pd.read_csv('trigram_col_clean.csv', encoding='utf-8')


# 3. 데이터 샘플 확인 (실제 값이 어떻게 생겼는지 확인용)
print("\n=== 2. 데이터 샘플 (상위 3개) ===")
display(df_check.head(3))


=== 2. 데이터 샘플 (상위 3개) ===


,ngram,freq,pmi
0,federal fund rate,12570,13.882325
1,federal reserve bank,12230,14.039774
2,open market committee,6632,12.646566


In [11]:
# 1. n-gram 파일 통합

ngram_files = [
    "bigram_col_clean.csv",
    "trigram_col_clean.csv",
    "fourgram_col_clean.csv",
    "fivegram_col_clean.csv"
]

target_vocab = set()

print(">>> 1. N-gram 파일 로드 및 통합 시작")

for f in ngram_files:
    try:
        df_temp = pd.read_csv(f)

        # 데이터 확인 결과 'ngram' 칼럼에 이미 "federal fund rate" 처럼 잘 들어가 있음
        if 'ngram' in df_temp.columns:
            # 혹시 모를 결측치만 제거하고 바로 리스트로 변환
            words = df_temp['ngram'].dropna().astype(str).tolist()
            target_vocab.update(words)
            print(f"   - {f}: {len(words)}개 단어 추가 완료")
        else:
            print(f"   ⚠️ {f} 파일에 'ngram' 칼럼이 없습니다.")

    except Exception as e:
        print(f"   ❌ {f} 로드 실패: {e}")

print(f"   => 총 분석 대상 단어 수: {len(target_vocab)}개")

>>> 1. N-gram 파일 로드 및 통합 시작
   - bigram_col_clean.csv: 17353개 단어 추가 완료
   - trigram_col_clean.csv: 32071개 단어 추가 완료
   - fourgram_col_clean.csv: 62843개 단어 추가 완료
   - fivegram_col_clean.csv: 98681개 단어 추가 완료
   => 총 분석 대상 단어 수: 210948개


In [13]:
# 2. 본문 데이터 로드 & 날짜 추출

print("\n>>> 2. 본문(Master) 로드 및 날짜 추출 중...")
master_file = "df_master_TM_proj_lemma.csv"
df_master = pd.read_csv(master_file)

# doc_id에서 날짜 추출 함수
def extract_date(doc_id):
    match = re.search(r"(20\d{6}|19\d{6})", str(doc_id)) # 20230501 형태 우선 검색
    if match:
        return pd.to_datetime(match.group(0), format='%Y%m%d')
    return None

# 날짜 칼럼 생성
df_master['DATE'] = df_master['doc_id'].apply(extract_date)

# 날짜를 못 찾았거나 본문이 없는 행은 제거
df_master = df_master.dropna(subset=['DATE', 'cleaned_text_lemma'])

print(f"   - 날짜가 식별된 문서 수: {len(df_master)}개")


>>> 2. 본문(Master) 로드 및 날짜 추출 중...
   - 날짜가 식별된 문서 수: 405959개


In [14]:
# effr 데이터 로드 & 라벨링

print("\n>>> 3. EFFR 데이터로 Hawkish/Dovish 라벨링 중...")

effr_file = "effr.csv"
df_effr = pd.read_csv(effr_file)

# 날짜 형식 통일
df_effr['DATE'] = pd.to_datetime(df_effr['DATE'])
df_effr = df_effr.sort_values('DATE')

rate_col = df_effr.columns[1] # effr 칼럼
print(f"   (참고: 금리 데이터 칼럼명은 '{rate_col}' 입니다)")

# 금리 변동폭(델타) 찾기 : 오늘 금리 - 어제 금리
df_effr['Delta'] = df_effr[rate_col].diff()

# 본문 데이터와 합치기 (날짜 기준 Inner Join)
df_train = pd.merge(df_master, df_effr[['DATE', 'Delta']], on='DATE', how='inner')

# 그룹 나누기
hawkish_docs = df_train[df_train['Delta'] > 0] # 금리 상승 (매파)
dovish_docs = df_train[df_train['Delta'] < 0]  # 금리 하락 (비둘기파)

# Delta == 0 은 제외

print(f"   - 분석 대상(매칭된) 문서: {len(df_train)}개")
print(f"   - Hawkish(인상) 문서: {len(hawkish_docs)}개")
print(f"   - Dovish(인하) 문서: {len(dovish_docs)}개")


>>> 3. EFFR 데이터로 Hawkish/Dovish 라벨링 중...
   (참고: 금리 데이터 칼럼명은 'EFFR' 입니다)
   - 분석 대상(매칭된) 문서: 259745개
   - Hawkish(인상) 문서: 32663개
   - Dovish(인하) 문서: 77432개


In [17]:
# 4. 단어 빈도 카운팅
# n-gram들이 매파 문서와 비둘기파 문서에서 각각 몇번 등장했는지 세는 작업
from sklearn.feature_extraction.text import CountVectorizer

print("\n>>> 4. 단어 빈도 카운팅 시작...")

# 우리가 만든 단어장(target_vocab)만 카운트
vectorizer = CountVectorizer(vocabulary=target_vocab, ngram_range=(2, 5))

# Hawkish 그룹 빈도 계산
X_hawkish = vectorizer.transform(hawkish_docs['cleaned_text_lemma'])
sum_hawkish = np.array(X_hawkish.sum(axis=0)).flatten()

# Dovish 그룹 빈도 계산
X_dovish = vectorizer.transform(dovish_docs['cleaned_text_lemma'])
sum_dovish = np.array(X_dovish.sum(axis=0)).flatten()

feature_names = vectorizer.get_feature_names_out()


>>> 4. 단어 빈도 카운팅 시작...


In [18]:
# 5. 점수 계산 및 json 변환
# 단어가 얼마나 매파적/비둘기파적인지 점수를 계산하는 과정

print(">>> 5. 극성 점수 계산 및 JSON 파일 생성 중...")

total_h = sum_hawkish.sum()
total_d = sum_dovish.sum()

market_lexicon = {}

for word, freq_h, freq_d in zip(feature_names, sum_hawkish, sum_dovish):
    # 1. 노이즈 제거 (양쪽 합쳐 5회 미만 등장 단어 제외)
    if (freq_h + freq_d) < 5:
        continue

    # 2. 확률 계산 (Smoothing 적용)
    prob_h = (freq_h + 1e-9) / total_h
    prob_d = (freq_d + 1e-9) / total_d

    # 3. 시장 접근법 공식 (Score calculation)
    # 범위: -1.0 ~ 1.0
    raw_score = (prob_h - prob_d) / (prob_h + prob_d)

    # 4. JSON 구조로 변환
    # polarity: hawkish/dovish
    # score: 점수의 절댓값(magnitude) 사용

    if raw_score > 0.2: # Hawkish 기준 (0.2는 임계값, 조정 가능)
        market_lexicon[word] = {
            "polarity": "hawkish",
            "score": round(abs(raw_score), 4)
        }
    elif raw_score < -0.2: # Dovish 기준
        market_lexicon[word] = {
            "polarity": "dovish",
            "score": round(abs(raw_score), 4)
        }
    # -0.2 ~ 0.2 사이인 '중립(Neutral)' 단어는 사전에 포함하지 않음

>>> 5. 극성 점수 계산 및 JSON 파일 생성 중...


In [19]:
output_file = "market_lexicon.json"

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(market_lexicon, f, indent=4, ensure_ascii=False)

print(f"\n✅ [성공] '{output_file}' 파일이 생성되었습니다.")
print(f"   - 총 등록된 단어 개수: {len(market_lexicon)}개")

# 확인용 출력 (상위 3개)
print("\n--- [생성된 사전 미리보기 (Top 3)] ---")
count = 0
for k, v in market_lexicon.items():
    print(f"'{k}': {v}")
    count += 1
    if count == 3: break


✅ [성공] 'market_lexicon.json' 파일이 생성되었습니다.
   - 총 등록된 단어 개수: 90948개

--- [생성된 사전 미리보기 (Top 3)] ---
'00 continue wednesday': {'polarity': 'hawkish', 'score': np.float64(0.7037)}
'00 pm': {'polarity': 'hawkish', 'score': np.float64(0.4146)}
'000 bank': {'polarity': 'hawkish', 'score': np.float64(0.2056)}
